# Imports

In [1]:
#export
import os
import sys
from pathlib import Path
p = '/home/sokolov/work/cycler/dHPO/exp/'
sys.path.append(p)
#sys.path.append(os.path.join(os.getcwd(),'exp'))

import time
import json
import numpy as np
from itertools import cycle
from functools import partial
from datetime import timedelta, datetime

from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
from airflow.utils.dates import days_ago


from nb_runner import cycle_exp, cycle_mutate, \
                      cycle_crossover, cycle_combine, cycle_all,\
                      bo_exp, bo_all
from config import cfg

# Code

## dag base

In [2]:
#export
d = days_ago(1)# + timedelta(hours=10, minutes=31)

default_args = {
    'owner': cfg.OWNER,
    'depends_on_past': False,
    #'start_date':d,
    'email': False,
    'email_on_failure': False,
    'email_on_retry': False,
    #'retries': 0,# overrided in pythonOperator down below
    #'retry_delay': timedelta(minutes=5),# overrided in pythonOperator down below
}

default_pool = cfg.DAG.DEF_POOL
#schedule_interval = cfg.DAG.SCHED_INTERVAL if cfg.DAG.SCHED_INTERVAL else None #@daily
description = cfg.DAG.DESC + '\n' + json.dumps(cfg, indent=4)

schedule_interval = '00 22 * * *'

dag = DAG(  cfg.DAG.NAME,
                max_active_runs=1,
                start_date=d,
                catchup=False,# for disabling backfill!
                description=description,
                schedule_interval=schedule_interval,
                default_args=default_args,
                )

def base_task_generator(name, func, dag, pool=default_pool, op_kwargs=None):
    task = PythonOperator(
                    task_id=name,
                    python_callable=func,
                    op_kwargs=op_kwargs,
                    provide_context=True,
                    pool=pool,
                    retries=cfg.DAG.RETRIES,
                    retry_delay=timedelta(minutes=cfg.DAG.RETRY_DELAY),
                    dag=dag,)
    return task
        
create_task = partial(base_task_generator, dag=dag)

def pull_results(**context):
    results = None
    if context is not None:
        task_ids = context['dag'].task_ids
        results = {task_id:context['ti'].xcom_pull(task_ids=task_id) for task_id in task_ids}
    return results


def upstream_results(**context):
    task = context['task']
    upstream_task_ids = task.upstream_task_ids
    rs = [pull_results(**context)[task_id] for task_id in upstream_task_ids]
    if not rs:
        rs = [{'configs':None, 'docker_results':None}]
    for result in rs:
        yield result        

def parse_pool(pool_str):
    pool_str = pool_str.split('_')[-1]
    return ','.join([i for i in pool_str if i.isdigit()])

def chunker_list(seq, size):
    return (seq[i::size] for i in range(size))

## dag extra

In [3]:
#export
def dw_cycle_param(func, **context):
    prev_results = next(upstream_results(**context))
    aux_cfg_files = prev_results['configs']
    gpus = parse_pool(context['ti'].pool)
    return func(aux_cfg_files=aux_cfg_files, gpus=gpus)

def dw_bo_param(func, **context):
    results = pull_results(**context)
    all_hp_points = []
    for task_id, r in results.items():
        if r:
            all_hp_points.append({'points':r['state'], 'target':r['docker_results']['metric']})  
    
    prev_results = next(upstream_results(**context))
    aux_cfg_files = prev_results['configs']
    gpus = parse_pool(context['ti'].pool)
    idx = cfg.GPUS.IDS.index(int(gpus))# wont work with single exp on multiple gpus!
    return func(aux_cfg_files=aux_cfg_files, gpus=gpus, hp_points=all_hp_points, idx=idx)

def dw_pooling(num=1, **context):
    key = 'metric'
    res = {}
    for r in upstream_results(**context):
        res[r['docker_results'][key]] = r['configs']
    res = sorted(res.items(), key=lambda x: x[0], reverse=True)
    res = res[:num]
    best_docker_results = [r[0] for r in res]
    best_configs = [r[1] for r in res]
    print(f'\n\tPooling: best result : {best_docker_results}, {best_configs}\n')
    return [{'configs':best_config} for best_config in best_configs]

def dw_dist(idx=None, **context):
    res = list(upstream_results(**context))[0][idx]
    return res

def distribute(name, num):
    tasks = []
    for i in range(num):
        task_name = f'distribute_{name}_{i}'
        tasks.append(create_task(task_name, dw_dist, op_kwargs={'idx':i}))
    return tasks
        
def block_optimize(n, name, func, dw_param):
    tasks = []
    gpus_avail = cycle(cfg.GPUS.IDS)
        
    for i in range(n):
        gpu = next(gpus_avail)
        task_name = f'{name}_{i}'
        pool = cfg.DAG.POOL_PREFIX + str(gpu)
        
        block_func = partial(func, seq_id=i, name=task_name)
        task = create_task(task_name, dw_param, pool=pool, op_kwargs={'func':block_func}) 
        tasks.append(task) 
    return tasks

# Tests

## random search

### chain

In [4]:
t1 = block_optimize(1, 'rs_exp', cycle_exp, dw_cycle_param)
t2 = block_optimize(1, 'rs_mut', cycle_mutate, dw_cycle_param)
t1[0] >> t2[0]

<Task(PythonOperator): rs_mut_0>

### parallel

In [5]:
block_optimize(2, 'rs_all', cycle_all, dw_cycle_param)

[<Task(PythonOperator): rs_all_0>, <Task(PythonOperator): rs_all_1>]

### parallel chains

In [6]:
tasks = {}
tasks['exp'] = block_optimize(3, 'cycle_e', cycle_exp, dw_cycle_param)
tasks['mut'] = block_optimize(3, 'cycle_m', cycle_mutate, dw_cycle_param)
for task_e, task_m in zip(tasks['exp'], tasks['mut']):
    task_e >> task_m

### pooling one

In [7]:
tasks = {}
tasks['exp'] = block_optimize(3, 'cycle_e', cycle_exp, dw_cycle_param)
pooling_task1 = create_task(f'pooling1', dw_pooling) 
tasks['exp'] >> pooling_task1

tasks['mut'] = block_optimize(5, 'cycle_m', cycle_mutate, dw_cycle_param)
pooling_task1 >> tasks['mut']

[<Task(PythonOperator): cycle_m_0>,
 <Task(PythonOperator): cycle_m_1>,
 <Task(PythonOperator): cycle_m_2>,
 <Task(PythonOperator): cycle_m_3>,
 <Task(PythonOperator): cycle_m_4>]

### pooling many

In [8]:
tasks = {}
#block_optimize(50, 'cycle_all', cycle_all, dw_cycle_param)
tasks['exp'] = block_optimize(3, 'cycle_e', cycle_exp, dw_cycle_param)
n_pooling = 2
pooling_task1 = create_task(f'pooling_{n_pooling}_best', partial(dw_pooling, num=n_pooling))
tasks['exp'] >> pooling_task1

dist_tasks = distribute('best_exp', n_pooling)
pooling_task1 >> dist_tasks

mut_tasks = block_optimize(5, 'cycle_m', cycle_mutate, dw_cycle_param)
for task_d, task_mut in zip(dist_tasks, chunker_list(mut_tasks, n_pooling)):
    task_d >> task_mut

### general one

In [9]:

tasks = {}
tasks['all'] = block_optimize(10, 'cycle_all', cycle_all, dw_cycle_param)

dist_num = 5
pooling_task1 = create_task(f'pooling_{dist_num}_best1', partial(dw_pooling, num=dist_num))
tasks['all'] >> pooling_task1

dist_tasks = distribute('best_all', dist_num)
pooling_task1 >> dist_tasks

tasks['mut'] = block_optimize(25, 'cycle_m', cycle_mutate, dw_cycle_param)
for dt, mt in zip(dist_tasks, chunker_list(tasks['mut'], dist_num)):
    dt >> mt

pooling_task2 = create_task(f'pooling_{dist_num}_best2', partial(dw_pooling, num=dist_num))
tasks['mut'] >> pooling_task2

dist_tasks2 = distribute('best_mut', dist_num)
pooling_task2 >> dist_tasks2


tasks['cr'] = block_optimize(10, 'cycle_cr', cycle_crossover, dw_cycle_param)
for dt, mt in zip(dist_tasks2, chunker_list(tasks['cr'], dist_num)):
    dt >> mt

pooling_task3 = create_task(f'pooling3', dw_pooling) 
tasks['cr'] >> pooling_task3

tasks['co'] = block_optimize(5, 'cycle_co', cycle_combine, dw_cycle_param)
pooling_task3 >> tasks['co']


[<Task(PythonOperator): cycle_co_0>,
 <Task(PythonOperator): cycle_co_1>,
 <Task(PythonOperator): cycle_co_2>,
 <Task(PythonOperator): cycle_co_3>,
 <Task(PythonOperator): cycle_co_4>]

## bo

In [10]:
#export
tasks_bo_all = block_optimize(80, 'bo_all', bo_all, dw_bo_param)

pooling_task1 = create_task(f'pooling1', dw_pooling) 
tasks_bo_all >> pooling_task1


<Task(PythonOperator): pooling1>

# More tests

In [11]:
#export
from airflow.models import TaskInstance
from datetime import datetime
import json
from pprint import pprint

In [12]:

def test1(**context):
    pprint(context)
    return 55

def test2(**context):
    pprint(context)
    pprint(pull_results2(**context))
    return 999


def pull_results2(**context):
    results = None
    if context is not None:
        task = context['task']
        dag = context['dag']
        task_ids = dag.task_ids
        #task_ids = task.upstream_task_ids
        for task_id in task_ids:
            print(task_id)
            xc = context['ti'].xcom_pull(task_ids=task_id)
            print(xc)
            
        results = {task_id:context['ti'].xcom_pull(task_ids=task_id) for task_id in task_ids}
    return results

task1 = base_task_generator('test1', test1, dag)
task2 = base_task_generator('test2', test2, dag)
#task1 >> task2


In [13]:
ti = TaskInstance(task=task2, execution_date=datetime.now())
ctxt = ti.get_template_context()
task2.execute(context=ctxt)

{'END_DATE': '2020-05-21',
 'conf': <airflow.configuration.AirflowConfigParser object at 0x7f87a38d02b0>,
 'dag': <DAG: dhpo2>,
 'dag_run': None,
 'ds': '2020-05-21',
 'ds_nodash': '20200521',
 'end_date': '2020-05-21',
 'execution_date': <Pendulum [2020-05-21T19:56:26.092923+00:00]>,
 'inlets': [],
 'latest_date': '2020-05-21',
 'macros': <module 'airflow.macros' from '/home/sokolov/.local/lib/python3.6/site-packages/airflow/macros/__init__.py'>,
 'next_ds': '2020-05-21',
 'next_ds_nodash': '20200521',
 'next_execution_date': <Pendulum [2020-05-21T22:00:00+00:00]>,
 'outlets': [],
 'params': {},
 'prev_ds': '2020-05-20',
 'prev_ds_nodash': '20200520',
 'prev_execution_date': <Pendulum [2020-05-20T22:00:00+00:00]>,
 'prev_execution_date_success': <Proxy at 0x7f878ab18cc8 with factory <function TaskInstance.get_template_context.<locals>.<lambda> at 0x7f878ab02488>>,
 'prev_start_date_success': <Proxy at 0x7f878ab18d08 with factory <function TaskInstance.get_template_context.<locals>.<la

999

In [14]:
d.task_ids

AttributeError: 'datetime.datetime' object has no attribute 'task_ids'

In [ ]:
d = ctxt['dag']
t = d.task_dict['test2']
t.xcom_pull(ctxt)

In [ ]:
task2.xcom_push(ctxt, 'test',13)

In [ ]:
task1.xcom_pull(ctxt)

In [ ]:
ti.pool, ti.current_state()

In [ ]:
ti = TaskInstance(task=task1, execution_date=datetime.now())
task1.execute(context=ti.get_template_context())
ti.pool, ti.current_state()

In [ ]:
ti.pool, ti.current_state()

In [ ]:
def task_state(args):
    dag = get_dag(args)
    task = dag.get_task(task_id=args.task_id)
    ti = TaskInstance(task, args.execution_date)
    print(ti.current_state())

# Export

In [15]:
!python3 extra/n2s.py dag.ipynb

Converted dag.ipynb to exp/nb_dag.py


In [16]:
!airflow list_dags

[2020-05-21 19:56:51,393] {__init__.py:51} INFO - Using executor LocalExecutor
[2020-05-21 19:56:51,394] {dagbag.py:396} INFO - Filling up the DagBag from /home/sokolov/airflow/dags
[2020-05-21 19:56:51,395] {dagbag.py:396} INFO - Filling up the DagBag from /home/sokolov/work/cycler/dHPO/exp


-------------------------------------------------------------------
DAGS
-------------------------------------------------------------------
dhpo2

